# Network construction

In this tutorial, we will see how to create and run a new EMERGENT network. We will also run this notebook from other tutorials to initialize a fresh network to play with. Before starting this tutorial, you should run the "Building blocks" tutorial to learn the purpose of the different nodes we construct here. First, let's create an empty network called "test":

In [ ]:
import os
path = '/emergent/emergent'
if os.name == 'posix':
    path = '~'+ path
%cd $path
%run utilities/new test

You should see a new folder called test in the emergent/networks/ directory. Next, we're going to define the core building blocks of an EMERGENT network: a Thing, a Hub, and a network declaration file. Each of these is typically its own .py file; here, we'll define them within the notebook and save them to files as we go using IPython's magic %%writefile command.

First, let's define a Thing: a driver
for some device in our experiment. Although Things are just standard Python classes which can contain any methods
you want, they should always have methods called \_actuate() and \_connect().

<div class="alert alert-block alert-info">
<b>_actuate():</b> takes a state dictionary as an argument. Overload this method according to the manufacturer API to 
send the state specified by the dictionary to the device to do something useful in the lab.
<br>
<b>_connect():</b> establishes a connection with the device and returns 1 if successful. Overload this method with 
the specific connection protocol required by the device, e.g. sending a start packet over TCP/IP.
</div>


Things also have standardized arguments which should be passed into the super().__init__() method as well:
<div class="alert alert-block alert-info">
<b>name:</b> the name which will be displayed in the experimental state dictionary and the GUI.
<br>
<b>params:</b> a dictionary containing any parameters you might want to pass to the device, like a serial number or 
analog input range.
<br>
<b>parent:</b> the Hub to which this Thing will be attached.
</div>

<div class="alert alert-block alert-warning">
If you plan on running EMERGENT across multiple computers, make sure to return early from the __init__ method
if Thing.local is False; this prevents nodes from being initialized on the wrong computer.
</div>

In [ ]:
%%writefile networks/test/things/test_thing.py 

from emergent.modules import Thing

class TestThing(Thing):
    ''' Thing driver for the virtual network in the 'basic' example. '''
    def __init__(self, name, params = {}, parent = None):
        ''' Register with the network and create two Input nodes, 'X' and 'Y'. '''
        super().__init__(name, parent, params = params)
        if not self.local:
            return

        for input_name in ['X', 'Y']:
            self.add_input(input_name)

    def _actuate(self, state):
        ''' Usually this method would change a physical state, but for our
            virtual network we only print the argument (note: the virtual state
            is updated within the public calling method Thing.actuate()).'''
        print('Actuating to', state)
        return

    def _connect(self):
        return 1

Now we will construct a Hub: a virtual construct which commands multiple things and measures some attached signal. 
The __init__ method should take the following arguments and pass them into the super().__init__ method:
<div class="alert alert-block alert-info">
<b>name:</b> the name which will be displayed in the experimental state dictionary and the GUI.
<br>
<b>params:</b> a dictionary containing any parameters you might want to pass to the hub.
<br>
<b>network:</b> the local cluster of Hubs to associate this instance with.
<br>
<b>addr:</b> the IP address of the PC where you want this Hub to run. The Hub will only be constructed if the 
PC has a network card matching this address. This allows Hubs across multiple decentralized PCs to be declared in
a single network.py file and selectively constructed depending on which PC is running the network.initialize() method.
</div>

In EMERGENT, experiments are written as methods of Hub classes with a standard call signature:
<div class="alert alert-block alert-info">
<b>state:</b> a dictionary specifying the state for which we want to run the experiment. For example, if we want
to measure a signal with a Thing named 'thing' set to coordinates X=1 and Y=2, we would pass {'thing': {'X':1, 'Y':2}}.
<br>
<b>params:</b> a dictionary containing any parameters you might want to pass to the experiment, e.g. an averaging
time.
</div>

The method is tagged with the @experiment decorator, which tells EMERGENT to treat it differently than a normal method: all tagged methods appear in spin-boxes in the GUI.

In [ ]:
%%writefile networks/test/hubs/test_hub.py 
import numpy as np
import time
from emergent.modules import Hub
from emergent.utilities.decorators import experiment, error

class TestHub(Hub):
    def __init__(self, name, params = {}, addr=None, network=None):
        super().__init__(name, addr=addr, network = network, params = params)
        self.options['Hello'] = lambda: print('Hello world!')
        
    def hello(self, name='world'):
        print('Hello', name)
        
    @experiment
    def gaussian(self, state, params = {'sigma_x': 0.3, 'sigma_y': 0.8, 'x0': 0.3, 'y0': 0.6, 'noise':0, 'delay': 0.5}):
        self.actuate(state)
        x=self.state['thing']['X']
        y=self.state['thing']['Y']
        x0 = params['x0']
        y0 = params['y0']
        sigma_x = params['sigma_x']
        sigma_y = params['sigma_y']
        power =  np.exp(-(x-x0)**2/sigma_x**2)*np.exp(-(y-y0)**2/sigma_y**2) + np.random.normal(0, params['noise'])

        return -power
    
    @error 
    def error_function(self, state, params={'setpoint': 1}):
        self.actuate(state)
        return self.state['thing']['X'] - params['setpoint']
    

The last step is to declare our objects in a network.py file, which should contain only a method called initialize() which is structured as follows:

In [ ]:
%%writefile networks/test/network.py

from emergent.networks.test.hubs.test_hub import TestHub
from emergent.networks.test.things.test_thing import TestThing
from emergent.modules.sequencing import Sequencer
from emergent.modules.switch import Switch

def initialize(network, params = {}):
    network.add_params(params)          # add the passed params to the network

    hub = TestHub(name='hub', network=network, addr='127.0.0.1')
    thing = TestThing('thing', params={}, parent=hub)
    
    hub.switches['A'] = Switch('A', {}, invert = False)
    hub.switches['B'] = Switch('B', {}, invert = False)
    steps = {'A_ON': {'duration': 1, 'state': {'A': 1, 'B': 0}},
             'B_ON': {'duration': 1, 'state': {'A': 0, 'B': 1}}}
    sequencer = Sequencer('sequencer', parent=hub, params={'sequence': steps})
    
    network.add_hub(hub)

Now let's run our network! First, execute the following cell to link the Qt event loop with IPython:

In [ ]:
%gui qt5

Now, run the launch() method in main.py, specifying the initialize method we defined above:

In [ ]:
%run main test --addr 127.0.0.1

You should see the main GUI for EMERGENT start up. Take some time to familiarize yourself with the various parts, or follow the tutorials below.

# State representation and actuation

The panel on the left side is called the Network Tree, and it displays all objects in your EMERGENT network. From this panel, you can check or update the state of any input. Notice that the X and Y inputs have initialized to 0, as there are no saved states on file yet for this new network. You can access the objects and state from the command-line using the global "network" variable, as in the following example:

In [ ]:
hub = network.hubs['hub']
thing = hub.children['thing']
X = thing.children['X']

print(hub.state)

To change a device state, you can simply double click on the corresponding input's "Value" entry in the Network panel, enter a new value, and press Return. This calls the hub.actuate(state) method, during which the hub distributes actuation commands to all affected devices. The devices themselves are updated through their \_actuate method as defined above, which should be overloaded in each Thing subclass.

Alternately, we can bypass the GUI and actuate from the command line:

In [ ]:
new_state = {'thing': {'X': 1, 'Y': 2}}
hub.actuate(new_state)
print('New state:', hub.state)